In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imageio.v3 as iio

import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

import torchdata.datapipes as dp 
from torchdata.datapipes import functional_datapipe
from torchdata.datapipes.iter import IterDataPipe

import pytorch_lightning as pl

cuda


In [2]:
DATA = Path("/media/sambhav/30AC4696AC46568E/datasets/building-footprints-dataset")
#DATA = Path("C:/Users/hp/Desktop/datasets/building-footprints-dataset")

TRAIN_DATA = DATA / "labelled" / "patches"
TRAIN_IMAGES = TRAIN_DATA / "images"
TRAIN_MASKS = TRAIN_DATA / "masks"
TRAIN_CATALOG = DATA / "labelled" / "metadata" / "patches.csv"

TEST_DATA = DATA / "labelled" / "scenes"
TEST_IMAGES = TEST_DATA / "images"
TEST_MASKS = TEST_DATA / "masks"

In [3]:
image_pipe = dp.iter.FileLister([TRAIN_IMAGES], non_deterministic=False)
mask_pipe =  dp.iter.FileLister([TRAIN_MASKS], non_deterministic=False)

In [4]:
print(next(iter(image_pipe)))
print(next(iter(mask_pipe)))

/media/sambhav/30AC4696AC46568E/datasets/building-footprints-dataset/labelled/patches/images/0000abf3-eb24-458e-b248-41016d2c25f8.jpg
/media/sambhav/30AC4696AC46568E/datasets/building-footprints-dataset/labelled/patches/masks/0000abf3-eb24-458e-b248-41016d2c25f8.jpg


In [5]:
def get_filename(file_path : str) -> str:
    return Path(file_path).name

train_pipe = dp.iter.IterKeyZipper(source_datapipe = image_pipe,
                                   ref_datapipe = mask_pipe,
                                   key_fn = get_filename,
                                   ref_key_fn = get_filename,
                                   keep_key = False)

In [6]:
next(iter(train_pipe))

('/media/sambhav/30AC4696AC46568E/datasets/building-footprints-dataset/labelled/patches/images/0000abf3-eb24-458e-b248-41016d2c25f8.jpg',
 '/media/sambhav/30AC4696AC46568E/datasets/building-footprints-dataset/labelled/patches/masks/0000abf3-eb24-458e-b248-41016d2c25f8.jpg')

In [7]:
TRAIN_CATALOG.as_posix()

'/media/sambhav/30AC4696AC46568E/datasets/building-footprints-dataset/labelled/metadata/patches.csv'

In [8]:
@functional_datapipe("get_bucket_url_pair")
class GetBucketURLPair(IterDataPipe):
    def __init__(self, source_datapipe: IterDataPipe, image_prefix:Path , mask_prefix:Path):
        self.source_datapipe = source_datapipe
        self.image_prefix = image_prefix
        self.mask_prefix = mask_prefix

    def __iter__(self):
        for file_name, dataset_name in self.source_datapipe:
            image_path = self.image_prefix / file_name 
            mask_path = self.mask_prefix / file_name
            yield (image_path, mask_path, dataset_name)

In [ ]:
#read_csv -> parse_csv -> image, mask -> get_transformed_image, mask

In [39]:
catalog = dp.iter.IterableWrapper([TRAIN_CATALOG.as_posix()])
catalog = catalog.open_files().parse_csv(skip_lines = 1)

next(iter(catalog))
##catalog = dp.iter.FileOpener()

'3fb79bc7-c7d4-45dd-8e4a-ac0f9d353ffd.jpg'

In [20]:
next(iter(catalog))

IsADirectoryError: [Errno 21] Is a directory: '/'